# Data Preparation & Preprocessing (MIMIC-IV Demo Dataset)
##### Project Author: Bruno Ferreira    
##### Date: March-April 2024

In this first notebook, we will focus on preparing and transforming the MIMIC-IV demo dataset into a suitable format to enable the planned sub-projects to be carried out.

#### Loading the dataset into a SQLite database, incorporating all the database's CSV files as tables

In [106]:
import sqlite3
import pandas as pd
from pathlib import Path

csv_directory = Path('C:/Users/bruno/Desktop/mimic') # Path to the directory containing CSV files

csv_files = csv_directory.glob('*.csv') # List of CSV files

conn = sqlite3.connect('C:/Users/bruno/Desktop/mimic/mimic_iv.db') # Create and connect to SQLite database

for csv_file in csv_files: # Iterate over CSV files and create corresponding tables in SQLite database
    table_name = csv_file.stem  # Extract table name from CSV file name
    df = pd.read_csv(csv_file, dtype=str)  # Read CSV file into DataFrame with all columns as strings
    df.to_sql(table_name, conn, if_exists='replace', index=False)  # Create SQLite table

conn.commit()

- Configuring the iPython-SQL environment for queries, and retrieving the total number of patients from the "patients" table for testing purposes

In [ ]:
!pip install ipython-sql

In [ ]:
%load_ext sql
%sql sqlite:///C:/Users/bruno/Desktop/mimic/mimic_iv.db

In [109]:
%sql SELECT COUNT(*) AS Total_Patients FROM patients

 * sqlite:///C:/Users/bruno/Desktop/mimic/mimic_iv.db
Done.


Total_Patients
100


- Selecting database tables with the most relevant information for the 3 subprojects

##### Relevant Tables:

`Admissions`: Provides information about each patient's hospital admission, including admission and discharge timestamps and the patient outcome (in-hospital death or not) which help understanding the patient's hospitalization journey.

`Patients`: Contains demographic information about patients, including its ID, age and gender, which can be used for patient stratification and risk assessment.

`Diagnoses_ICD`: Includes diagnostic codes assigned to patients during their hospital stay, which are valuable for identifying underlying health conditions and comorbidities that may influence patient outcomes.

`Procedures_ICD`: Provides information about procedures performed on patients during their hospitalization, which can be relevant for understanding the severity of illness and predicting outcomes.

`Prescriptions`: Contains data about medications prescribed to patients, including dosage, frequency, and route of administration, which can be important for assessing treatment regimens and potential medication-related adverse events.

`omr`: Offers miscellaneous health information such as blood pressure, height, weight, and body mass index, which can be useful for risk assessment and outcome prediction.

`labevents`: Includes laboratory test results, which are crucial for assessing patient health status, monitoring disease progression, and identifying abnormalities that may indicate adverse events or mortality risk.

#### Data wrangling

- Let's first inspect the raw data in the tables, total number of entries (rows) and columns, name and data type of each column and number of non-null values in each column:

In [110]:
import numpy as np 

relevant_tables = ['admissions', 'patients', 'diagnoses_icd', 'procedures_icd', 'prescriptions', 'omr', 'labevents']

# Define a function to preprocess each table
def inspect_tables(table_name):
 
    query = f"SELECT * FROM {table_name};" # Fetch data from the table
    data = %sql $query

    df = data.DataFrame() # Convert data to a pandas DataFrame

    print(f"\n{'='*20} Inspecting {table_name} {'='*20}") # Print header for table being processed

    print(f"\nGlimpse of {table_name} DataFrame:") # Display glimpse of DataFrame
    print(df.head())

    print(f"\nSummary information of {table_name} DataFrame:") # Display number of rows and columns, missing values and variable data types
    print(df.info())

# Iterate over relevant tables and preprocess each one
for table_name in relevant_tables:
    inspect_tables(table_name)

 * sqlite:///C:/Users/bruno/Desktop/mimic/mimic_iv.db
Done.

==================== Inspecting admissions ====================

Glimpse of admissions DataFrame:
  subject_id   hadm_id            admittime            dischtime  \
0   10004235  24181354  2196-02-24 14:38:00  2196-03-04 14:02:00   
1   10009628  25926192  2153-09-17 17:08:00  2153-09-25 13:20:00   
2   10018081  23983182  2134-08-18 02:02:00  2134-08-23 19:35:00   
3   10006053  22942076  2111-11-13 23:39:00  2111-11-15 17:20:00   
4   10031404  21606243  2113-08-04 18:46:00  2113-08-06 20:57:00   

             deathtime admission_type admit_provider_id  \
0                 None         URGENT            P03YMR   
1                 None         URGENT            P41R5N   
2                 None         URGENT            P233F6   
3  2111-11-15 17:20:00         URGENT            P38TI6   
4                 None         URGENT            P07HDB   

       admission_location        discharge_location insurance language  \
0  

- Let's load the tables information into pandas dataframes to to make them easier to handle

In [111]:
for table in relevant_tables:
    query = f"SELECT * FROM {table}"
    globals()[table] = pd.read_sql_query(query, conn)

- Cleaning and pre-processing the raw tables

`Admissions table`

In [112]:
admissions_del = ['hadm_id','admit_provider_id','admission_type','insurance','language','marital_status','edregtime','edouttime','deathtime','discharge_location']
admissions.drop(columns=admissions_del, inplace=True) # Drops unnecessary columns

admissions['admittime'] = pd.to_datetime(admissions['admittime']) # Converts variables to correct data type
admissions['dischtime'] = pd.to_datetime(admissions['dischtime']) # P.S.: Date years were changed to de-identify patients but are consistent across the databases tables for each patient.
admissions['hospital_expire_flag'] = admissions['hospital_expire_flag'].astype(int)

headers = ['subject_id','admittime','dischtime','admission_location','race','deceased'] # Assigns new column names
admissions.columns = headers

if admissions.isnull().sum().sum() == 0: # Checks for missing values
    print("Relevant columns from this table have no missing values.")
else:
    print("One or more relevant columns from this table have missing values.")

Relevant columns from this table have no missing values.


`Patients table`

In [113]:
patients_del = ['anchor_year', 'anchor_year_group'] # Drops unnecessary columns
patients.drop(columns=patients_del, inplace=True) 

patients['anchor_age'] = patients['anchor_age'].astype(int) # Converts variables to correct data type
patients['dod'] = pd.to_datetime(patients['dod'])

headers = ['subject_id', 'gender', 'age', 'dateofdeath'] # Assigns new column names
patients.columns = headers

if patients.isnull().sum().sum() == 0: # Checks for missing values (P.S.: Null values in 'dateofdeath' mean "No death", so none are missing values.)
    print("Relevant columns from this table have no missing values.")
else:
    print("One or more relevant columns from this table have missing values.")

One or more relevant columns from this table have missing values.


`Diagnoses_ICD table`

In [114]:
diagnoses_icd.drop(columns=['hadm_id'], inplace=True)  # Drops unnecessary columns

diagnoses_icd['seq_num'] = diagnoses_icd['seq_num'].astype(int) # Converts variable to correct data type

diagnoses_icd.rename(columns={'seq_num': 'diagnosis_order'}, inplace=True) # Renames 'seq_num' column

if diagnoses_icd.isnull().sum().sum() == 0: # Checks for missing values
    print("Relevant columns from this table have no missing values.")
else:
    print("One or more relevant columns from this table have missing values.")

Relevant columns from this table have no missing values.


`Procedures_ICD table`

In [115]:
procedures_icd.drop(columns=['hadm_id'], inplace=True) # Drops unnecessary columns

procedures_icd.rename(columns={'chartdate': 'procedure_date', 'seq_num': 'procedure_order'}, inplace=True) # Renames columns

procedures_icd['procedure_order'] = procedures_icd['procedure_order'].astype(int) # Converts variable to correct data type

if procedures_icd.isnull().sum().sum() == 0: # Checks for missing values
    print("Relevant columns from this table have no missing values.")
else:
    print("One or more relevant columns from this table have missing values.")

Relevant columns from this table have no missing values.


`Prescriptions table`

In [116]:
prescriptions = prescriptions[['subject_id', 'drug']] # Keeps most relevant columns

prescriptions.rename(columns={'drug': 'prescriptions'}) # Renames 'drug' column to "prescriptions"

if prescriptions.isnull().sum().sum() == 0: # Checks for missing values
    print("Relevant columns from this table have no missing values.")
else:
    print("One or more relevant columns from this table have missing values.")

Relevant columns from this table have no missing values.


`OMR (Miscellaneous measurements) table`

In [117]:
omr.drop(columns=['seq_num'], inplace=True) # Drops unnecessary column

omr.rename(columns={'chartdate': 'date', 'result_name': 'measurement', 'result_value': 'value'}, inplace=True) # Renames columns

omr['date'] = pd.to_datetime(omr['date']) # Converts variable to correct data type

### Splits blood pressure values into systolic and diastolic columns
### omr[['systolic_bp', 'diastolic_bp']] = omr['value'].str.split('/', expand=True)

if omr.isnull().sum().sum() == 0: # Checks for missing values
    print("Relevant columns from this table have no missing values.")
else:
    print("One or more relevant columns from this table have missing values.")
#FEATURE ENGINNERING: AGGREGATE (MEAN VALUES). SEPARATE MEASUREMENTS IN COLUMNS

Relevant columns from this table have no missing values.


`Labevents (Laboratory Results) table`

In [118]:
labevents = labevents[['subject_id', 'valuenum', 'flag']] # Keeps only the required columns

labevents.rename(columns={'flag': 'labresult'}, inplace=True) # Renames column

labevents.dropna(subset=['valuenum'], inplace=True) # Filters rows without missing lab result values

labevents.drop(columns=['valuenum'], inplace=True) # Removes lab result values column (no longer necessary)

labevents.isnull().sum() # Checks for missing values
#FEATURE ENGINNERING: ex: % of abnormal lab results(?)

subject_id        0
labresult     56340
dtype: int64

- Missing values on 'labresult' mean "normal" lab results, so let's replace them accordingly:

In [119]:
labevents.loc[labevents['labresult'].isnull(), 'labresult'] = 'normal'

- Lastly, we need to remove duplicated rows that don't add value to our research (Excluding 'admissions' and 'labevents' tables, because duplicates like multiple abnormal lab results or multiple admissions for the same patient may give us valuable insights):

In [120]:
# Duplicated patients and identical diagnoses, procedures, prescriptions and measurements are redundant
duplicates_from = ['patients', 'diagnoses_icd', 'procedures_icd', 'prescriptions', 'omr']

for table_name in duplicates_from:
    df = globals()[table_name]  # Get DataFrame by its name
    
    if df.duplicated().any(): # Check for duplicates
        print(f"Duplicates found in {table_name}. Removing duplicates...")
        df.drop_duplicates(inplace=True)  # Remove duplicates
        print(f"Duplicates removed from {table_name}.")
    else:
        print(f"No duplicates found in {table_name}.")

No duplicates found in patients.
Duplicates found in diagnoses_icd. Removing duplicates...
Duplicates removed from diagnoses_icd.
No duplicates found in procedures_icd.
Duplicates found in prescriptions. Removing duplicates...
Duplicates removed from prescriptions.
Duplicates found in omr. Removing duplicates...
Duplicates removed from omr.


#### Feature Engineering

In [121]:
admissions.head()

,subject_id,admittime,dischtime,admission_location,race,deceased
0,10004235,2196-02-24 14:38:00,2196-03-04 14:02:00,TRANSFER FROM HOSPITAL,BLACK/CAPE VERDEAN,0
1,10009628,2153-09-17 17:08:00,2153-09-25 13:20:00,TRANSFER FROM HOSPITAL,HISPANIC/LATINO - PUERTO RICAN,0
2,10018081,2134-08-18 02:02:00,2134-08-23 19:35:00,TRANSFER FROM HOSPITAL,WHITE,0
3,10006053,2111-11-13 23:39:00,2111-11-15 17:20:00,TRANSFER FROM HOSPITAL,UNKNOWN,1
4,10031404,2113-08-04 18:46:00,2113-08-06 20:57:00,TRANSFER FROM HOSPITAL,WHITE,0


In [122]:
diagnoses_icd.head()

,subject_id,diagnosis_order,icd_code,icd_version
0,10035185,3,4139,9
1,10035185,10,V707,9
2,10035185,1,41401,9
3,10035185,9,3899,9
4,10035185,11,V8532,9


In [123]:
labevents.head()

,subject_id,labresult
0,10014354,normal
1,10014354,abnormal
2,10014354,abnormal
3,10014354,abnormal
4,10014354,abnormal


In [124]:
omr.head()

,subject_id,date,measurement,value
0,10011398,2146-12-01,Height (Inches),63
1,10011398,2147-01-22,Weight (Lbs),127
2,10011398,2146-12-01,Weight (Lbs),135
3,10011398,2147-07-24,Weight (Lbs),136
4,10011398,2147-03-26,Weight (Lbs),136


In [125]:
patients.head()

,subject_id,gender,age,dateofdeath
0,10014729,F,21,NaT
1,10003400,F,72,2137-09-02
2,10002428,F,80,NaT
3,10032725,F,38,2143-03-30
4,10027445,F,48,2146-02-09


In [126]:
prescriptions.head()

,subject_id,drug
0,10027602,Fentanyl Citrate
2,10027602,Lorazepam
3,10027602,Midazolam
5,10023239,Insulin Pump (Self Administering Medication)
8,10027602,Propofol


In [127]:
procedures_icd.head()

,subject_id,procedure_order,procedure_date,icd_code,icd_version
0,10011398,3,2146-12-15,3961,9
1,10011398,2,2146-12-15,3615,9
2,10011398,1,2146-12-15,3614,9
3,10014729,4,2125-03-23,3897,9
4,10014729,1,2125-03-20,3403,9
